# Computational Graph

## Simple Graph with One Parameter
Let's start with simple computation graph. Consider composite function $f(x)$ as follows:

$$f(x) = ((x + 5)^3+1)^2$$

And let's compute our function $f(x)$ at the point $x = -3$. So, if we decompose our function $f(x)$ into simple functions, we get:

$$h= x + 5 = 2$$
$$g = h^3 + 1 = 9$$
$$f = g^2 = 81$$

See [Computational Graph]() for more details.

We'll use PyTorch to compute the output (forward propagation) of our function $f(x)$ at the point $x = -3$. Then we will compute the gradients (backward propagation) of our function $f(x)$ with respect to $x$.

Note: We use PyTorch in this example to merely utilize its differentitation features and compute the gradients. This is completely optional and you can compute the gradients manually or using other libraries.

In [1]:
import torch

In [2]:
x = torch.tensor(-3.0, requires_grad=True)

Since we want to compute the gradient (derivative) of our function $f(x)$ with respect to $x$, we need to set `requires_grad=True` for any parameter that will be part of the computation graph for which we want to compute the gradient.

In [3]:
f_x = ((x + 5) ** 3 + 1) ** 2
print(f_x.data)

tensor(81.)


In [4]:
f_x.backward()

# Gradients
print(f"df/dx: {x.grad}")

df/dx: 216.0


During the forward propagation, PyTorch automatically builds a directed acyclic graph (DAG) of operations. This is when we calculate `f_x`. PyTorch create this graph when `torch.tensor` with `requires_grad=True` is involved in the computation.

When we call `f_x.backward()`, it walks backward through the computational graph to to each parameter (leaf node) and apply the chain rule of calculus to compute the gradients of the output (the final node where we called `backward()`) with respect to each parameter.

Gradient Accumulation: The calculated gradients are stored in the `grad` attribute of each parameter tensor that has `requires_grad=True`.

This [PyTorch Computation Graph](https://www.youtube.com/watch?v=MswxJw-8PvE) is a good video on this topic.

## Graph with Multiple Parameters

Now let's consider a simple linear regression model with one input feature $x$ and one target $y$. The model is defined as follows:

$$f_{w,b}(x) = wx + b$$

Where $w$ is the weight and $b$ is the bias. 

The [cost function]() is defined as the mean squared error (MSE) for a dataset with only one sample:

$$J(w,b) = \frac{1}{2}(f_{w,b}(x) - y)^2$$


[Gradient Descent]() is the most common optimization algorithm used to minimize the cost function. The algorithm works by iteratively updating the parameters in the opposite direction of the gradients of the cost function with respect to the parameters. So, we need to compute the gradients of the cost function with respect to the parameters $w$ and $b$.
 

See [Computational Graph]() for more details.


Let's define our input $x$, target value $y$, weight $w$ and bias $b$ as follows:

In [5]:
x = torch.tensor(-3.0)
y = torch.tensor(5.0)
w = torch.tensor(3.0, requires_grad=True)
b = torch.tensor(1.0, requires_grad=True)

We didn't set the `requires_grad=True` for $x$ and $y$ because they are the input and target values. They are not parameters of the model which we need to optimize by computing the gradients.

### Step 1 - Forward Propagation (Compute Model Output)
We'll compute the model's output

In [6]:
# Forward Propagation (compute the output)
c = w * x
f = c + b

### Step 2 - Compute the Cost Function

In [7]:
# Compute the Cost
d = f - y
J = (d**2) / 2

In PyTorch, the intermediate gradients for non-leaf nodes are not stored by default which is more efficient in practice when we have a large number of parameters. However, in this example to demonstrate the backward steps in the computational graph, we will change this behavior by calling `retain_grad()` on the intermediate tensors.

In [8]:
c.retain_grad()
f.retain_grad()
d.retain_grad()

In [9]:
print(f"Input feature x and target y:\nx: {x.data}\ny: {y.data}\n")
print(f"Model parameters:\nw: {w.data}\nb: {b.data}\n")
print(f"Model output:\nc: {c.data}\nf: {f.data}\n")
print(f"Cost:\nd: {d.data}\nJ: {J.data}\n")

Input feature x and target y:
x: -3.0
y: 5.0

Model parameters:
w: 3.0
b: 1.0

Model output:
c: -9.0
f: -8.0

Cost:
d: -13.0
J: 84.5



### Step 3 - Backpropagation (Compute the Gradients)

In [10]:
# Backpropagation (compute the gradients)
J.backward()

In [11]:
print(f"dJ/dd: {d.grad}")
print(f"dJ/df: {f.grad}")
print(f"dJ/dc: {c.grad}")
print(f"dJ/db: {b.grad}")
print(f"dJ/dw: {w.grad}")

dJ/dd: -13.0
dJ/df: -13.0
dJ/dc: -13.0
dJ/db: -13.0
dJ/dw: 39.0
